In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,EarlyStopping

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_mean_ang(df):
    res = []
    for i, row in df.iterrows():
        if row['inc_angle'] != 'na':
            ang = float(row['inc_angle'])
            res.append(ang)
    return np.mean(res)

ang_mean = get_mean_ang(train_df)
print(ang_mean)

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        
        if row['inc_angle'] == 'na':
            ang = -1
        else:
            ang = float(row['inc_angle'])
        
        band_3 = (band_1 - band_2)*ang/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<30:
        return 0.0005
    elif epoch<70:
        return 0.0001
    else:
        return 0.00005

y = train_df.is_iceberg.values
print(y[:5])

Using TensorFlow backend.


39.2687074779
(1604, 75, 75, 3) (8424, 75, 75, 3)
[0 0 1 0 0]


In [4]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding='same',input_shape=(75,75,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model
print('model model')


model model


In [5]:

    

def kfold_train(fold_cnt=3,rnd=252):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.63601, saving model to best_m.h5
 - 2s - loss: 0.6948 - acc: 0.5487 - val_loss: 0.6360 - val_acc: 0.6511
Epoch 2/120
Epoch 00002: val_loss improved from 0.63601 to 0.53226, saving model to best_m.h5
 - 2s - loss: 0.5708 - acc: 0.6773 - val_loss: 0.5323 - val_acc: 0.7508
Epoch 3/120
Epoch 00003: val_loss improved from 0.53226 to 0.45900, saving model to best_m.h5
 - 2s - loss: 0.5145 - acc: 0.7224 - val_loss: 0.4590 - val_acc: 0.8006
Epoch 4/120
Epoch 00004: val_loss improved from 0.45900 to 0.42928, saving model to best_m.h5
 - 2s - loss: 0.4794 - acc: 0.7607 - val_loss: 0.4293 - val_acc: 0.8255
Epoch 5/120
Epoch 00005: val_loss did not improve
 - 1s - loss: 0.4335 - acc: 0.7888 - val_loss: 0.4551 - val_acc: 0.7695
Epoch 6/120
Epoch 00006: val_loss improved from 0.42928 to 0.36624, saving model to best_m.h5
 - 2s - loss: 0.4000 - acc: 0.8070 - val_loss: 0.3662 - val_acc: 0.8411
Epoch 7/120
Epoch 00007: val_loss improved from 0.36

Epoch 61/120
Epoch 00061: val_loss did not improve
 - 1s - loss: 0.1978 - acc: 0.9133 - val_loss: 0.2029 - val_acc: 0.9190
Epoch 62/120
Epoch 00062: val_loss did not improve
 - 1s - loss: 0.2077 - acc: 0.9094 - val_loss: 0.2042 - val_acc: 0.9065
Epoch 63/120
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.1906 - acc: 0.9195 - val_loss: 0.2070 - val_acc: 0.9097
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.1898 - acc: 0.9164 - val_loss: 0.2019 - val_acc: 0.9159
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.2067 - acc: 0.9141 - val_loss: 0.2164 - val_acc: 0.9190
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.2068 - acc: 0.9115 - val_loss: 0.2122 - val_acc: 0.9034
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.1896 - acc: 0.9219 - val_loss: 0.2033 - val_acc: 0.9159
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.1925 - acc: 0.9141 - val_loss: 0.2112 - val_acc: 0.9065
Epoch 69/120
Epo

Epoch 5/120
Epoch 00005: val_loss improved from 0.43948 to 0.42410, saving model to best_m.h5
 - 2s - loss: 0.4892 - acc: 0.7537 - val_loss: 0.4241 - val_acc: 0.7819
Epoch 6/120
Epoch 00006: val_loss improved from 0.42410 to 0.41700, saving model to best_m.h5
 - 2s - loss: 0.4094 - acc: 0.8123 - val_loss: 0.4170 - val_acc: 0.8037
Epoch 7/120
Epoch 00007: val_loss improved from 0.41700 to 0.39161, saving model to best_m.h5
 - 2s - loss: 0.3884 - acc: 0.8180 - val_loss: 0.3916 - val_acc: 0.8318
Epoch 8/120
Epoch 00008: val_loss improved from 0.39161 to 0.29525, saving model to best_m.h5
 - 2s - loss: 0.3594 - acc: 0.8286 - val_loss: 0.2953 - val_acc: 0.8660
Epoch 9/120
Epoch 00009: val_loss did not improve
 - 2s - loss: 0.3597 - acc: 0.8344 - val_loss: 0.3351 - val_acc: 0.8318
Epoch 10/120
Epoch 00010: val_loss improved from 0.29525 to 0.26584, saving model to best_m.h5
 - 2s - loss: 0.3586 - acc: 0.8328 - val_loss: 0.2658 - val_acc: 0.8785
Epoch 11/120
Epoch 00011: val_loss did not impr

Epoch 66/120
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.2047 - acc: 0.9070 - val_loss: 0.1792 - val_acc: 0.9315
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.2039 - acc: 0.9125 - val_loss: 0.1696 - val_acc: 0.9408
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.1884 - acc: 0.9070 - val_loss: 0.1904 - val_acc: 0.9315
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.2099 - acc: 0.9141 - val_loss: 0.1856 - val_acc: 0.9377
Epoch 70/120
Epoch 00070: val_loss did not improve
 - 1s - loss: 0.2044 - acc: 0.9125 - val_loss: 0.1928 - val_acc: 0.9252
Epoch 71/120
Epoch 00071: val_loss did not improve
 - 1s - loss: 0.2023 - acc: 0.9125 - val_loss: 0.1878 - val_acc: 0.9315
Epoch 72/120
Epoch 00072: val_loss did not improve
 - 1s - loss: 0.1870 - acc: 0.9172 - val_loss: 0.1908 - val_acc: 0.9315
Epoch 73/120
Epoch 00073: val_loss did not improve
 - 1s - loss: 0.1789 - acc: 0.9266 - val_loss: 0.1770 - val_acc: 0.9408
Epoch 74/120
Epo

Epoch 10/120
Epoch 00010: val_loss improved from 0.34480 to 0.28637, saving model to best_m.h5
 - 2s - loss: 0.3345 - acc: 0.8490 - val_loss: 0.2864 - val_acc: 0.8754
Epoch 11/120
Epoch 00011: val_loss did not improve
 - 2s - loss: 0.3320 - acc: 0.8398 - val_loss: 0.3094 - val_acc: 0.8754
Epoch 12/120
Epoch 00012: val_loss improved from 0.28637 to 0.27456, saving model to best_m.h5
 - 2s - loss: 0.3251 - acc: 0.8625 - val_loss: 0.2746 - val_acc: 0.9003
Epoch 13/120
Epoch 00013: val_loss did not improve
 - 2s - loss: 0.2981 - acc: 0.8656 - val_loss: 0.2773 - val_acc: 0.8910
Epoch 14/120
Epoch 00014: val_loss did not improve
 - 1s - loss: 0.3107 - acc: 0.8503 - val_loss: 0.3217 - val_acc: 0.8660
Epoch 15/120
Epoch 00015: val_loss did not improve
 - 1s - loss: 0.3391 - acc: 0.8476 - val_loss: 0.2968 - val_acc: 0.8847
Epoch 16/120
Epoch 00016: val_loss did not improve
 - 2s - loss: 0.2872 - acc: 0.8750 - val_loss: 0.2753 - val_acc: 0.8972
Epoch 17/120
Epoch 00017: val_loss did not improve


Epoch 74/120
Epoch 00074: val_loss did not improve
 - 2s - loss: 0.1797 - acc: 0.9383 - val_loss: 0.2371 - val_acc: 0.9065
Epoch 75/120
Epoch 00075: val_loss did not improve
 - 1s - loss: 0.1795 - acc: 0.9219 - val_loss: 0.2484 - val_acc: 0.9034
Epoch 76/120
Epoch 00076: val_loss did not improve
 - 2s - loss: 0.1733 - acc: 0.9281 - val_loss: 0.2482 - val_acc: 0.9034
Epoch 77/120
Epoch 00077: val_loss did not improve
 - 1s - loss: 0.1863 - acc: 0.9258 - val_loss: 0.2354 - val_acc: 0.9097
Epoch 78/120
Epoch 00078: val_loss did not improve
 - 1s - loss: 0.1679 - acc: 0.9375 - val_loss: 0.2306 - val_acc: 0.9065
Epoch 79/120
Epoch 00079: val_loss did not improve
 - 2s - loss: 0.1976 - acc: 0.9164 - val_loss: 0.2270 - val_acc: 0.9065
Epoch 80/120
Epoch 00080: val_loss did not improve
 - 2s - loss: 0.1781 - acc: 0.9305 - val_loss: 0.2602 - val_acc: 0.9003
Epoch 81/120
Epoch 00081: val_loss did not improve
 - 1s - loss: 0.1763 - acc: 0.9320 - val_loss: 0.2370 - val_acc: 0.9065
Epoch 82/120
Epo

Epoch 00016: val_loss improved from 0.23282 to 0.20946, saving model to best_m.h5
 - 2s - loss: 0.2816 - acc: 0.8757 - val_loss: 0.2095 - val_acc: 0.9221
Epoch 17/120
Epoch 00017: val_loss did not improve
 - 2s - loss: 0.2613 - acc: 0.8779 - val_loss: 0.2427 - val_acc: 0.8941
Epoch 18/120
Epoch 00018: val_loss did not improve
 - 1s - loss: 0.2780 - acc: 0.8750 - val_loss: 0.2432 - val_acc: 0.8816
Epoch 19/120
Epoch 00019: val_loss did not improve
 - 1s - loss: 0.2812 - acc: 0.8781 - val_loss: 0.3088 - val_acc: 0.8567
Epoch 20/120
Epoch 00020: val_loss did not improve
 - 2s - loss: 0.2941 - acc: 0.8638 - val_loss: 0.2573 - val_acc: 0.8847
Epoch 21/120
Epoch 00021: val_loss did not improve
 - 1s - loss: 0.2556 - acc: 0.8797 - val_loss: 0.3228 - val_acc: 0.8692
Epoch 22/120
Epoch 00022: val_loss did not improve
 - 1s - loss: 0.2627 - acc: 0.8852 - val_loss: 0.2138 - val_acc: 0.9003
Epoch 23/120
Epoch 00023: val_loss did not improve
 - 1s - loss: 0.2577 - acc: 0.8820 - val_loss: 0.2127 - v

Epoch 79/120
Epoch 00079: val_loss did not improve
 - 1s - loss: 0.1833 - acc: 0.9203 - val_loss: 0.1620 - val_acc: 0.9439
Epoch 80/120
Epoch 00080: val_loss did not improve
 - 1s - loss: 0.1927 - acc: 0.9242 - val_loss: 0.1713 - val_acc: 0.9377
Epoch 81/120
Epoch 00081: val_loss did not improve
 - 1s - loss: 0.1883 - acc: 0.9185 - val_loss: 0.1677 - val_acc: 0.9502
Epoch 82/120
Epoch 00082: val_loss did not improve
 - 1s - loss: 0.1958 - acc: 0.9201 - val_loss: 0.1620 - val_acc: 0.9564
Epoch 83/120
Epoch 00083: val_loss did not improve
 - 1s - loss: 0.1875 - acc: 0.9281 - val_loss: 0.1640 - val_acc: 0.9502
Epoch 84/120
Epoch 00084: val_loss did not improve
 - 2s - loss: 0.1695 - acc: 0.9266 - val_loss: 0.1617 - val_acc: 0.9439
Epoch 85/120
Epoch 00085: val_loss did not improve
 - 2s - loss: 0.1904 - acc: 0.9154 - val_loss: 0.1694 - val_acc: 0.9408
Epoch 86/120
Epoch 00086: val_loss did not improve
 - 1s - loss: 0.1791 - acc: 0.9297 - val_loss: 0.1642 - val_acc: 0.9502
Epoch 87/120
Epo

Epoch 22/120
Epoch 00022: val_loss did not improve
 - 1s - loss: 0.2771 - acc: 0.8813 - val_loss: 0.2672 - val_acc: 0.8750
Epoch 23/120
Epoch 00023: val_loss improved from 0.24269 to 0.23154, saving model to best_m.h5
 - 2s - loss: 0.2427 - acc: 0.8898 - val_loss: 0.2315 - val_acc: 0.8781
Epoch 24/120
Epoch 00024: val_loss did not improve
 - 2s - loss: 0.2560 - acc: 0.8883 - val_loss: 0.2819 - val_acc: 0.8656
Epoch 25/120
Epoch 00025: val_loss did not improve
 - 1s - loss: 0.2699 - acc: 0.8774 - val_loss: 0.2470 - val_acc: 0.8844
Epoch 26/120
Epoch 00026: val_loss did not improve
 - 1s - loss: 0.2421 - acc: 0.8890 - val_loss: 0.2365 - val_acc: 0.8844
Epoch 27/120
Epoch 00027: val_loss did not improve
 - 1s - loss: 0.2505 - acc: 0.8945 - val_loss: 0.2437 - val_acc: 0.8781
Epoch 28/120
Epoch 00028: val_loss did not improve
 - 1s - loss: 0.2363 - acc: 0.8969 - val_loss: 0.2439 - val_acc: 0.8812
Epoch 29/120
Epoch 00029: val_loss did not improve
 - 1s - loss: 0.2365 - acc: 0.8969 - val_los

Epoch 87/120
Epoch 00087: val_loss did not improve
 - 2s - loss: 0.1851 - acc: 0.9195 - val_loss: 0.2453 - val_acc: 0.8969
Epoch 88/120
Epoch 00088: val_loss did not improve
 - 2s - loss: 0.1760 - acc: 0.9211 - val_loss: 0.2120 - val_acc: 0.8938
Epoch 89/120
Epoch 00089: val_loss did not improve
 - 2s - loss: 0.1781 - acc: 0.9203 - val_loss: 0.2453 - val_acc: 0.8969
Epoch 90/120
Epoch 00090: val_loss did not improve
 - 1s - loss: 0.1817 - acc: 0.9242 - val_loss: 0.2365 - val_acc: 0.9000
Epoch 91/120
Epoch 00091: val_loss did not improve
 - 2s - loss: 0.2061 - acc: 0.9109 - val_loss: 0.2215 - val_acc: 0.9062
Epoch 92/120
Epoch 00092: val_loss did not improve
 - 2s - loss: 0.1797 - acc: 0.9289 - val_loss: 0.2318 - val_acc: 0.8969
Epoch 93/120
Epoch 00093: val_loss did not improve
 - 1s - loss: 0.1743 - acc: 0.9289 - val_loss: 0.2289 - val_acc: 0.9031
Epoch 94/120
Epoch 00094: val_loss improved from 0.21175 to 0.21040, saving model to best_m.h5
 - 2s - loss: 0.1987 - acc: 0.9196 - val_los

In [6]:
import pickle
with open('../2nd_features/cnn_1_aug_1_fold5_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))



0.186282422491


In [7]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(32, (3, 3),input_shape=(75, 75, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    return model
print('model model')

model model


In [8]:
def kfold_train(fold_cnt=3,rnd=252):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.51742, saving model to best_m.h5
 - 2s - loss: 0.6157 - acc: 0.6276 - val_loss: 0.5174 - val_acc: 0.7632
Epoch 2/120
Epoch 00002: val_loss improved from 0.51742 to 0.43530, saving model to best_m.h5
 - 1s - loss: 0.5150 - acc: 0.7443 - val_loss: 0.4353 - val_acc: 0.8224
Epoch 3/120
Epoch 00003: val_loss improved from 0.43530 to 0.36521, saving model to best_m.h5
 - 1s - loss: 0.4085 - acc: 0.8133 - val_loss: 0.3652 - val_acc: 0.8224
Epoch 4/120
Epoch 00004: val_loss improved from 0.36521 to 0.31350, saving model to best_m.h5
 - 1s - loss: 0.3785 - acc: 0.8334 - val_loss: 0.3135 - val_acc: 0.8505
Epoch 5/120
Epoch 00005: val_loss did not improve
 - 1s - loss: 0.3274 - acc: 0.8615 - val_loss: 0.3941 - val_acc: 0.8100
Epoch 6/120
Epoch 00006: val_loss improved from 0.31350 to 0.29271, saving model to best_m.h5
 - 1s - loss: 0.3200 - acc: 0.8555 - val_loss: 0.2927 - val_acc: 0.8660
Epoch 7/120
Epoch 00007: val_loss did not improve
 -

Epoch 61/120
Epoch 00061: val_loss did not improve
 - 1s - loss: 0.1630 - acc: 0.9328 - val_loss: 0.2018 - val_acc: 0.9128
Epoch 62/120
Epoch 00062: val_loss did not improve
 - 1s - loss: 0.1696 - acc: 0.9289 - val_loss: 0.2090 - val_acc: 0.9159
Epoch 63/120
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.1421 - acc: 0.9461 - val_loss: 0.2091 - val_acc: 0.9128
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.1450 - acc: 0.9406 - val_loss: 0.2199 - val_acc: 0.9097
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.1667 - acc: 0.9258 - val_loss: 0.1945 - val_acc: 0.9097
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.1551 - acc: 0.9383 - val_loss: 0.2028 - val_acc: 0.9159
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.1795 - acc: 0.9224 - val_loss: 0.2104 - val_acc: 0.9097
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.1446 - acc: 0.9398 - val_loss: 0.2140 - val_acc: 0.9065
Epoch 69/120
Epo

Epoch 7/120
Epoch 00007: val_loss improved from 0.26608 to 0.22630, saving model to best_m.h5
 - 1s - loss: 0.3259 - acc: 0.8602 - val_loss: 0.2263 - val_acc: 0.9065
Epoch 8/120
Epoch 00008: val_loss did not improve
 - 1s - loss: 0.3448 - acc: 0.8544 - val_loss: 0.2268 - val_acc: 0.9159
Epoch 9/120
Epoch 00009: val_loss did not improve
 - 1s - loss: 0.2837 - acc: 0.8766 - val_loss: 0.2399 - val_acc: 0.8785
Epoch 10/120
Epoch 00010: val_loss improved from 0.22630 to 0.20611, saving model to best_m.h5
 - 1s - loss: 0.3165 - acc: 0.8630 - val_loss: 0.2061 - val_acc: 0.9128
Epoch 11/120
Epoch 00011: val_loss improved from 0.20611 to 0.18551, saving model to best_m.h5
 - 1s - loss: 0.2836 - acc: 0.8734 - val_loss: 0.1855 - val_acc: 0.9221
Epoch 12/120
Epoch 00012: val_loss did not improve
 - 1s - loss: 0.2340 - acc: 0.9031 - val_loss: 0.2047 - val_acc: 0.9097
Epoch 13/120
Epoch 00013: val_loss did not improve
 - 1s - loss: 0.2561 - acc: 0.8795 - val_loss: 0.2359 - val_acc: 0.8910
Epoch 14/1

Epoch 70/120
Epoch 00070: val_loss did not improve
 - 1s - loss: 0.1553 - acc: 0.9344 - val_loss: 0.1490 - val_acc: 0.9439
Epoch 71/120
Epoch 00071: val_loss did not improve
 - 1s - loss: 0.1472 - acc: 0.9453 - val_loss: 0.1415 - val_acc: 0.9502
Epoch 72/120
Epoch 00072: val_loss did not improve
 - 1s - loss: 0.1576 - acc: 0.9391 - val_loss: 0.1427 - val_acc: 0.9470
Epoch 73/120
Epoch 00073: val_loss did not improve
 - 1s - loss: 0.1390 - acc: 0.9469 - val_loss: 0.1418 - val_acc: 0.9470
Epoch 74/120
Epoch 00074: val_loss did not improve
 - 1s - loss: 0.1605 - acc: 0.9266 - val_loss: 0.1411 - val_acc: 0.9470
Epoch 75/120
Epoch 00075: val_loss did not improve
 - 1s - loss: 0.1443 - acc: 0.9453 - val_loss: 0.1453 - val_acc: 0.9408
Epoch 76/120
Epoch 00076: val_loss did not improve
 - 1s - loss: 0.1432 - acc: 0.9500 - val_loss: 0.1414 - val_acc: 0.9470
Epoch 77/120
Epoch 00077: val_loss did not improve
 - 1s - loss: 0.1625 - acc: 0.9357 - val_loss: 0.1414 - val_acc: 0.9470
Epoch 78/120
Epo

Epoch 00013: val_loss did not improve
 - 1s - loss: 0.2849 - acc: 0.8701 - val_loss: 0.2691 - val_acc: 0.8785
Epoch 14/120
Epoch 00014: val_loss did not improve
 - 1s - loss: 0.2415 - acc: 0.8914 - val_loss: 0.2562 - val_acc: 0.8941
Epoch 15/120
Epoch 00015: val_loss did not improve
 - 1s - loss: 0.3026 - acc: 0.8664 - val_loss: 0.2885 - val_acc: 0.8723
Epoch 16/120
Epoch 00016: val_loss improved from 0.25461 to 0.23459, saving model to best_m.h5
 - 1s - loss: 0.2549 - acc: 0.8906 - val_loss: 0.2346 - val_acc: 0.8910
Epoch 17/120
Epoch 00017: val_loss did not improve
 - 1s - loss: 0.2604 - acc: 0.8834 - val_loss: 0.3173 - val_acc: 0.8505
Epoch 18/120
Epoch 00018: val_loss improved from 0.23459 to 0.22749, saving model to best_m.h5
 - 1s - loss: 0.2466 - acc: 0.8880 - val_loss: 0.2275 - val_acc: 0.9003
Epoch 19/120
Epoch 00019: val_loss did not improve
 - 1s - loss: 0.2447 - acc: 0.8898 - val_loss: 0.3063 - val_acc: 0.8567
Epoch 20/120
Epoch 00020: val_loss did not improve
 - 1s - loss:

Epoch 77/120
Epoch 00077: val_loss did not improve
 - 1s - loss: 0.1463 - acc: 0.9430 - val_loss: 0.2359 - val_acc: 0.9034
Epoch 78/120
Epoch 00078: val_loss did not improve
 - 1s - loss: 0.1424 - acc: 0.9406 - val_loss: 0.2323 - val_acc: 0.9003
Epoch 79/120
Epoch 00079: val_loss did not improve
 - 1s - loss: 0.1509 - acc: 0.9351 - val_loss: 0.2432 - val_acc: 0.8879
Epoch 80/120
Epoch 00080: val_loss did not improve
 - 1s - loss: 0.1599 - acc: 0.9312 - val_loss: 0.2278 - val_acc: 0.9034
Epoch 81/120
Epoch 00081: val_loss did not improve
 - 1s - loss: 0.1511 - acc: 0.9398 - val_loss: 0.2345 - val_acc: 0.8941
Epoch 82/120
Epoch 00082: val_loss did not improve
 - 1s - loss: 0.1571 - acc: 0.9281 - val_loss: 0.2256 - val_acc: 0.9065
Epoch 83/120
Epoch 00083: val_loss did not improve
 - 1s - loss: 0.1591 - acc: 0.9326 - val_loss: 0.2315 - val_acc: 0.9065
Epoch 84/120
Epoch 00084: val_loss did not improve
 - 1s - loss: 0.1647 - acc: 0.9281 - val_loss: 0.2258 - val_acc: 0.9065
Epoch 85/120
Epo

Epoch 00019: val_loss improved from 0.20002 to 0.19185, saving model to best_m.h5
 - 1s - loss: 0.2490 - acc: 0.8920 - val_loss: 0.1918 - val_acc: 0.9315
Epoch 20/120
Epoch 00020: val_loss did not improve
 - 1s - loss: 0.2267 - acc: 0.9117 - val_loss: 0.2553 - val_acc: 0.8941
Epoch 21/120
Epoch 00021: val_loss did not improve
 - 1s - loss: 0.2445 - acc: 0.9000 - val_loss: 0.2640 - val_acc: 0.8910
Epoch 22/120
Epoch 00022: val_loss did not improve
 - 1s - loss: 0.2277 - acc: 0.9047 - val_loss: 0.2920 - val_acc: 0.8660
Epoch 23/120
Epoch 00023: val_loss did not improve
 - 1s - loss: 0.2554 - acc: 0.8937 - val_loss: 0.2112 - val_acc: 0.9377
Epoch 24/120
Epoch 00024: val_loss improved from 0.19185 to 0.17515, saving model to best_m.h5
 - 1s - loss: 0.2392 - acc: 0.8945 - val_loss: 0.1752 - val_acc: 0.9252
Epoch 25/120
Epoch 00025: val_loss did not improve
 - 1s - loss: 0.2178 - acc: 0.9086 - val_loss: 0.2285 - val_acc: 0.9159
Epoch 26/120
Epoch 00026: val_loss did not improve
 - 1s - loss:

Epoch 81/120
Epoch 00081: val_loss did not improve
 - 1s - loss: 0.1407 - acc: 0.9383 - val_loss: 0.1513 - val_acc: 0.9502
Epoch 82/120
Epoch 00082: val_loss did not improve
 - 1s - loss: 0.1358 - acc: 0.9484 - val_loss: 0.1535 - val_acc: 0.9439
Epoch 83/120
Epoch 00083: val_loss did not improve
 - 1s - loss: 0.1410 - acc: 0.9453 - val_loss: 0.1501 - val_acc: 0.9533
Epoch 84/120
Epoch 00084: val_loss did not improve
 - 1s - loss: 0.1444 - acc: 0.9344 - val_loss: 0.1487 - val_acc: 0.9377
Epoch 85/120
Epoch 00085: val_loss did not improve
 - 1s - loss: 0.1463 - acc: 0.9359 - val_loss: 0.1449 - val_acc: 0.9502
Epoch 86/120
Epoch 00086: val_loss did not improve
 - 1s - loss: 0.1284 - acc: 0.9500 - val_loss: 0.1453 - val_acc: 0.9533
Epoch 87/120
Epoch 00087: val_loss did not improve
 - 1s - loss: 0.1444 - acc: 0.9373 - val_loss: 0.1528 - val_acc: 0.9502
Epoch 88/120
Epoch 00088: val_loss did not improve
 - 1s - loss: 0.1452 - acc: 0.9375 - val_loss: 0.1461 - val_acc: 0.9533
Epoch 89/120
Epo

Epoch 24/120
Epoch 00024: val_loss did not improve
 - 1s - loss: 0.2096 - acc: 0.9070 - val_loss: 0.2885 - val_acc: 0.8969
Epoch 25/120
Epoch 00025: val_loss did not improve
 - 1s - loss: 0.2231 - acc: 0.8992 - val_loss: 0.3030 - val_acc: 0.8781
Epoch 26/120
Epoch 00026: val_loss did not improve
 - 1s - loss: 0.2235 - acc: 0.9086 - val_loss: 0.3152 - val_acc: 0.8906
Epoch 27/120
Epoch 00027: val_loss did not improve
 - 1s - loss: 0.2029 - acc: 0.9141 - val_loss: 0.2834 - val_acc: 0.8719
Epoch 28/120
Epoch 00028: val_loss did not improve
 - 1s - loss: 0.2078 - acc: 0.9148 - val_loss: 0.2799 - val_acc: 0.9000
Epoch 29/120
Epoch 00029: val_loss did not improve
 - 1s - loss: 0.1985 - acc: 0.9133 - val_loss: 0.3353 - val_acc: 0.8844
Epoch 30/120
Epoch 00030: val_loss improved from 0.23850 to 0.23794, saving model to best_m.h5
 - 1s - loss: 0.2028 - acc: 0.9187 - val_loss: 0.2379 - val_acc: 0.9125
Epoch 31/120
Epoch 00031: val_loss did not improve
 - 1s - loss: 0.1821 - acc: 0.9234 - val_los

Epoch 91/120
Epoch 00091: val_loss did not improve
 - 1s - loss: 0.1304 - acc: 0.9453 - val_loss: 0.2828 - val_acc: 0.8906
Epoch 92/120
Epoch 00092: val_loss did not improve
 - 1s - loss: 0.1304 - acc: 0.9453 - val_loss: 0.2870 - val_acc: 0.9062
Epoch 93/120
Epoch 00093: val_loss did not improve
 - 1s - loss: 0.1349 - acc: 0.9390 - val_loss: 0.2685 - val_acc: 0.9094
Epoch 94/120
Epoch 00094: val_loss did not improve
 - 1s - loss: 0.1420 - acc: 0.9391 - val_loss: 0.3011 - val_acc: 0.8906
Epoch 95/120
Epoch 00095: val_loss did not improve
 - 1s - loss: 0.1315 - acc: 0.9398 - val_loss: 0.2779 - val_acc: 0.8938
Epoch 96/120
Epoch 00096: val_loss did not improve
 - 1s - loss: 0.1508 - acc: 0.9437 - val_loss: 0.2704 - val_acc: 0.8969
Epoch 97/120
Epoch 00097: val_loss did not improve
 - 1s - loss: 0.1408 - acc: 0.9445 - val_loss: 0.2767 - val_acc: 0.8906
Epoch 98/120
Epoch 00098: val_loss did not improve
 - 1s - loss: 0.1266 - acc: 0.9484 - val_loss: 0.2737 - val_acc: 0.8969
Epoch 99/120
Epo

In [9]:
with open('../2nd_features/cnn_2_aug_1_fold5_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))



0.182247202008


In [10]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')

model model


In [11]:
def kfold_train(fold_cnt=3,rnd=252):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.62045, saving model to best_m.h5
 - 3s - loss: 0.6920 - acc: 0.5370 - val_loss: 0.6204 - val_acc: 0.6355
Epoch 2/120
Epoch 00002: val_loss improved from 0.62045 to 0.58945, saving model to best_m.h5
 - 2s - loss: 0.6039 - acc: 0.6255 - val_loss: 0.5894 - val_acc: 0.7321
Epoch 3/120
Epoch 00003: val_loss improved from 0.58945 to 0.47348, saving model to best_m.h5
 - 2s - loss: 0.5129 - acc: 0.7445 - val_loss: 0.4735 - val_acc: 0.7788
Epoch 4/120
Epoch 00004: val_loss did not improve
 - 2s - loss: 0.4350 - acc: 0.7992 - val_loss: 0.4742 - val_acc: 0.7632
Epoch 5/120
Epoch 00005: val_loss did not improve
 - 2s - loss: 0.4370 - acc: 0.7919 - val_loss: 0.5925 - val_acc: 0.6791
Epoch 6/120
Epoch 00006: val_loss improved from 0.47348 to 0.37312, saving model to best_m.h5
 - 2s - loss: 0.3994 - acc: 0.8148 - val_loss: 0.3731 - val_acc: 0.8287
Epoch 7/120
Epoch 00007: val_loss did not improve
 - 2s - loss: 0.3568 - acc: 0.8326 - val_loss:

Epoch 60/120
Epoch 00060: val_loss improved from 0.20138 to 0.19445, saving model to best_m.h5
 - 2s - loss: 0.1881 - acc: 0.9226 - val_loss: 0.1945 - val_acc: 0.9346
Epoch 61/120
Epoch 00061: val_loss did not improve
 - 2s - loss: 0.1821 - acc: 0.9219 - val_loss: 0.2021 - val_acc: 0.9221
Epoch 62/120
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.1747 - acc: 0.9266 - val_loss: 0.2084 - val_acc: 0.9221
Epoch 63/120
Epoch 00063: val_loss did not improve
 - 2s - loss: 0.1753 - acc: 0.9320 - val_loss: 0.2034 - val_acc: 0.9190
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.1793 - acc: 0.9328 - val_loss: 0.2136 - val_acc: 0.9190
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.1838 - acc: 0.9273 - val_loss: 0.2033 - val_acc: 0.9283
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.1642 - acc: 0.9326 - val_loss: 0.1968 - val_acc: 0.9283
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.1989 - acc: 0.9219 - val_los

Epoch 5/120
Epoch 00005: val_loss improved from 0.42034 to 0.36713, saving model to best_m.h5
 - 2s - loss: 0.4091 - acc: 0.8156 - val_loss: 0.3671 - val_acc: 0.8349
Epoch 6/120
Epoch 00006: val_loss improved from 0.36713 to 0.36216, saving model to best_m.h5
 - 2s - loss: 0.4257 - acc: 0.7904 - val_loss: 0.3622 - val_acc: 0.8224
Epoch 7/120
Epoch 00007: val_loss improved from 0.36216 to 0.30000, saving model to best_m.h5
 - 2s - loss: 0.3956 - acc: 0.8193 - val_loss: 0.3000 - val_acc: 0.9003
Epoch 8/120
Epoch 00008: val_loss did not improve
 - 2s - loss: 0.3679 - acc: 0.8359 - val_loss: 0.3332 - val_acc: 0.8629
Epoch 9/120
Epoch 00009: val_loss improved from 0.30000 to 0.25569, saving model to best_m.h5
 - 2s - loss: 0.3679 - acc: 0.8380 - val_loss: 0.2557 - val_acc: 0.8816
Epoch 10/120
Epoch 00010: val_loss improved from 0.25569 to 0.24867, saving model to best_m.h5
 - 2s - loss: 0.3304 - acc: 0.8513 - val_loss: 0.2487 - val_acc: 0.8972
Epoch 11/120
Epoch 00011: val_loss did not impr

Epoch 68/120
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.1963 - acc: 0.9211 - val_loss: 0.1667 - val_acc: 0.9315
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.1764 - acc: 0.9289 - val_loss: 0.1578 - val_acc: 0.9315
Epoch 70/120
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.1892 - acc: 0.9234 - val_loss: 0.1504 - val_acc: 0.9377
Epoch 71/120
Epoch 00071: val_loss did not improve
 - 2s - loss: 0.1785 - acc: 0.9248 - val_loss: 0.1706 - val_acc: 0.9252
Epoch 72/120
Epoch 00072: val_loss did not improve
 - 2s - loss: 0.1937 - acc: 0.9195 - val_loss: 0.1642 - val_acc: 0.9315
Epoch 73/120
Epoch 00073: val_loss did not improve
 - 2s - loss: 0.1709 - acc: 0.9271 - val_loss: 0.1733 - val_acc: 0.9283
Epoch 74/120
Epoch 00074: val_loss did not improve
 - 2s - loss: 0.2004 - acc: 0.9154 - val_loss: 0.1644 - val_acc: 0.9315
Epoch 75/120
Epoch 00075: val_loss did not improve
 - 2s - loss: 0.1867 - acc: 0.9154 - val_loss: 0.1586 - val_acc: 0.9346
Epoch 76/120
Epo

Epoch 11/120
Epoch 00011: val_loss improved from 0.34247 to 0.28467, saving model to best_m.h5
 - 2s - loss: 0.3039 - acc: 0.8781 - val_loss: 0.2847 - val_acc: 0.8723
Epoch 12/120
Epoch 00012: val_loss did not improve
 - 2s - loss: 0.2849 - acc: 0.8656 - val_loss: 0.3692 - val_acc: 0.8474
Epoch 13/120
Epoch 00013: val_loss improved from 0.28467 to 0.27250, saving model to best_m.h5
 - 2s - loss: 0.3107 - acc: 0.8703 - val_loss: 0.2725 - val_acc: 0.8754
Epoch 14/120
Epoch 00014: val_loss did not improve
 - 2s - loss: 0.2750 - acc: 0.8670 - val_loss: 0.2879 - val_acc: 0.8847
Epoch 15/120
Epoch 00015: val_loss improved from 0.27250 to 0.27243, saving model to best_m.h5
 - 2s - loss: 0.3026 - acc: 0.8617 - val_loss: 0.2724 - val_acc: 0.8941
Epoch 16/120
Epoch 00016: val_loss did not improve
 - 2s - loss: 0.2862 - acc: 0.8630 - val_loss: 0.2727 - val_acc: 0.8879
Epoch 17/120
Epoch 00017: val_loss improved from 0.27243 to 0.27063, saving model to best_m.h5
 - 2s - loss: 0.2642 - acc: 0.8797 

Epoch 73/120
Epoch 00073: val_loss did not improve
 - 2s - loss: 0.1879 - acc: 0.9230 - val_loss: 0.2051 - val_acc: 0.9221
Epoch 74/120
Epoch 00074: val_loss did not improve
 - 2s - loss: 0.1733 - acc: 0.9234 - val_loss: 0.2083 - val_acc: 0.9128
Epoch 75/120
Epoch 00075: val_loss did not improve
 - 2s - loss: 0.1847 - acc: 0.9289 - val_loss: 0.2192 - val_acc: 0.9065
Epoch 76/120
Epoch 00076: val_loss did not improve
 - 2s - loss: 0.1826 - acc: 0.9281 - val_loss: 0.2055 - val_acc: 0.9190
Epoch 77/120
Epoch 00077: val_loss did not improve
 - 2s - loss: 0.1721 - acc: 0.9209 - val_loss: 0.2173 - val_acc: 0.9159
Epoch 78/120
Epoch 00078: val_loss did not improve
 - 2s - loss: 0.1681 - acc: 0.9312 - val_loss: 0.2130 - val_acc: 0.9159
Epoch 79/120
Epoch 00079: val_loss did not improve
 - 2s - loss: 0.1680 - acc: 0.9359 - val_loss: 0.2251 - val_acc: 0.9034
Epoch 80/120
Epoch 00080: val_loss did not improve
 - 2s - loss: 0.1781 - acc: 0.9297 - val_loss: 0.2229 - val_acc: 0.9128
Epoch 81/120
Epo

Epoch 00016: val_loss did not improve
 - 2s - loss: 0.2704 - acc: 0.8891 - val_loss: 0.2745 - val_acc: 0.8910
Epoch 17/120
Epoch 00017: val_loss improved from 0.24041 to 0.23945, saving model to best_m.h5
 - 2s - loss: 0.3187 - acc: 0.8612 - val_loss: 0.2395 - val_acc: 0.9034
Epoch 18/120
Epoch 00018: val_loss improved from 0.23945 to 0.20443, saving model to best_m.h5
 - 2s - loss: 0.2760 - acc: 0.8758 - val_loss: 0.2044 - val_acc: 0.9159
Epoch 19/120
Epoch 00019: val_loss did not improve
 - 2s - loss: 0.3076 - acc: 0.8693 - val_loss: 0.2119 - val_acc: 0.9159
Epoch 20/120
Epoch 00020: val_loss improved from 0.20443 to 0.19116, saving model to best_m.h5
 - 2s - loss: 0.2528 - acc: 0.8943 - val_loss: 0.1912 - val_acc: 0.9221
Epoch 21/120
Epoch 00021: val_loss did not improve
 - 2s - loss: 0.2554 - acc: 0.8812 - val_loss: 0.2191 - val_acc: 0.9097
Epoch 22/120
Epoch 00022: val_loss did not improve
 - 2s - loss: 0.2730 - acc: 0.8867 - val_loss: 0.2127 - val_acc: 0.9097
Epoch 23/120
Epoch 0

Epoch 79/120
Epoch 00079: val_loss did not improve
 - 2s - loss: 0.2032 - acc: 0.9195 - val_loss: 0.1609 - val_acc: 0.9346
Epoch 80/120
Epoch 00080: val_loss did not improve
 - 2s - loss: 0.1776 - acc: 0.9312 - val_loss: 0.1593 - val_acc: 0.9346
Epoch 81/120
Epoch 00081: val_loss did not improve
 - 2s - loss: 0.1958 - acc: 0.9177 - val_loss: 0.1575 - val_acc: 0.9315
Epoch 82/120
Epoch 00082: val_loss did not improve
 - 2s - loss: 0.1703 - acc: 0.9281 - val_loss: 0.1573 - val_acc: 0.9346
Epoch 83/120
Epoch 00083: val_loss did not improve
 - 2s - loss: 0.1821 - acc: 0.9250 - val_loss: 0.1583 - val_acc: 0.9315
Epoch 84/120
Epoch 00084: val_loss did not improve
 - 2s - loss: 0.1781 - acc: 0.9289 - val_loss: 0.1602 - val_acc: 0.9315
Epoch 85/120
Epoch 00085: val_loss did not improve
 - 2s - loss: 0.1887 - acc: 0.9273 - val_loss: 0.1612 - val_acc: 0.9377
Epoch 86/120
Epoch 00086: val_loss did not improve
 - 2s - loss: 0.1873 - acc: 0.9154 - val_loss: 0.1634 - val_acc: 0.9315
Epoch 87/120
Epo

Epoch 20/120
Epoch 00020: val_loss did not improve
 - 2s - loss: 0.2404 - acc: 0.8953 - val_loss: 0.3008 - val_acc: 0.8812
Epoch 21/120
Epoch 00021: val_loss did not improve
 - 2s - loss: 0.2748 - acc: 0.8781 - val_loss: 0.2863 - val_acc: 0.8812
Epoch 22/120
Epoch 00022: val_loss did not improve
 - 2s - loss: 0.2431 - acc: 0.9000 - val_loss: 0.2536 - val_acc: 0.9187
Epoch 23/120
Epoch 00023: val_loss did not improve
 - 2s - loss: 0.2569 - acc: 0.8875 - val_loss: 0.2676 - val_acc: 0.8906
Epoch 24/120
Epoch 00024: val_loss improved from 0.25309 to 0.25306, saving model to best_m.h5
 - 2s - loss: 0.2509 - acc: 0.8961 - val_loss: 0.2531 - val_acc: 0.8969
Epoch 25/120
Epoch 00025: val_loss did not improve
 - 2s - loss: 0.2265 - acc: 0.9047 - val_loss: 0.2537 - val_acc: 0.9094
Epoch 26/120
Epoch 00026: val_loss did not improve
 - 2s - loss: 0.2523 - acc: 0.8961 - val_loss: 0.2579 - val_acc: 0.9094
Epoch 27/120
Epoch 00027: val_loss did not improve
 - 2s - loss: 0.2374 - acc: 0.8976 - val_los

Epoch 85/120
Epoch 00085: val_loss did not improve
 - 2s - loss: 0.1726 - acc: 0.9297 - val_loss: 0.2503 - val_acc: 0.9125
Epoch 86/120
Epoch 00086: val_loss did not improve
 - 2s - loss: 0.1707 - acc: 0.9305 - val_loss: 0.2377 - val_acc: 0.9125
Epoch 87/120
Epoch 00087: val_loss did not improve
 - 2s - loss: 0.1841 - acc: 0.9180 - val_loss: 0.2417 - val_acc: 0.9156
Epoch 88/120
Epoch 00088: val_loss did not improve
 - 2s - loss: 0.1603 - acc: 0.9297 - val_loss: 0.2449 - val_acc: 0.9094
Epoch 89/120
Epoch 00089: val_loss did not improve
 - 2s - loss: 0.1481 - acc: 0.9476 - val_loss: 0.2588 - val_acc: 0.9062
Epoch 90/120
Epoch 00090: val_loss did not improve
 - 2s - loss: 0.1612 - acc: 0.9383 - val_loss: 0.2377 - val_acc: 0.9094
Epoch 91/120
Epoch 00091: val_loss did not improve
 - 2s - loss: 0.1583 - acc: 0.9328 - val_loss: 0.2428 - val_acc: 0.9125
Epoch 92/120
Epoch 00092: val_loss did not improve
 - 2s - loss: 0.1540 - acc: 0.9289 - val_loss: 0.2730 - val_acc: 0.9062
Epoch 93/120
Epo

In [12]:
with open('../2nd_features/cnn_3_aug_1_fold5_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))



0.178629762882


In [13]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def ConvBlock(model, layers, filters):
    '''Create [layers] layers consisting of zero padding, a convolution with [filters] 3x3 filters and batch normalization. Perform max pooling after the last layer.'''
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
        model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()

    # Input image: 75x75x3
    model.add(Lambda(lambda x: x, input_shape=(75, 75, 3)))
    ConvBlock(model, 1, 32)
    # 37x37x32
    ConvBlock(model, 1, 64)
    # 18x18x64
    ConvBlock(model, 1, 128)
    # 9x9x128
    ConvBlock(model, 1, 128)
    # 4x4x128
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))
    
    return model
print('model model')


model model


In [14]:
def kfold_train(fold_cnt=3,rnd=252):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.52787, saving model to best_m.h5
 - 4s - loss: 0.6171 - acc: 0.7253 - val_loss: 0.5279 - val_acc: 0.7757
Epoch 2/120
Epoch 00002: val_loss improved from 0.52787 to 0.37529, saving model to best_m.h5
 - 3s - loss: 0.3772 - acc: 0.8279 - val_loss: 0.3753 - val_acc: 0.8505
Epoch 3/120
Epoch 00003: val_loss did not improve
 - 3s - loss: 0.3889 - acc: 0.8142 - val_loss: 0.5575 - val_acc: 0.7726
Epoch 4/120
Epoch 00004: val_loss improved from 0.37529 to 0.27311, saving model to best_m.h5
 - 3s - loss: 0.3827 - acc: 0.8320 - val_loss: 0.2731 - val_acc: 0.8723
Epoch 5/120
Epoch 00005: val_loss did not improve
 - 2s - loss: 0.2772 - acc: 0.8890 - val_loss: 0.3387 - val_acc: 0.8255
Epoch 6/120
Epoch 00006: val_loss did not improve
 - 2s - loss: 0.2831 - acc: 0.8758 - val_loss: 0.2855 - val_acc: 0.8692
Epoch 7/120
Epoch 00007: val_loss improved from 0.27311 to 0.26413, saving model to best_m.h5
 - 2s - loss: 0.2978 - acc: 0.8706 - val_loss:

Epoch 62/120
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.1377 - acc: 0.9430 - val_loss: 0.2022 - val_acc: 0.9128
Epoch 63/120
Epoch 00063: val_loss did not improve
 - 2s - loss: 0.1315 - acc: 0.9500 - val_loss: 0.2107 - val_acc: 0.9159
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.1445 - acc: 0.9422 - val_loss: 0.2328 - val_acc: 0.9065
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.1603 - acc: 0.9422 - val_loss: 0.2126 - val_acc: 0.9128
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.1317 - acc: 0.9539 - val_loss: 0.2128 - val_acc: 0.9065
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.1326 - acc: 0.9477 - val_loss: 0.2155 - val_acc: 0.9159
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.1414 - acc: 0.9375 - val_loss: 0.2254 - val_acc: 0.9159
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.1345 - acc: 0.9477 - val_loss: 0.2105 - val_acc: 0.9097
Epoch 70/120
Epo

Epoch 7/120
Epoch 00007: val_loss did not improve
 - 2s - loss: 0.2722 - acc: 0.8789 - val_loss: 0.2867 - val_acc: 0.8692
Epoch 8/120
Epoch 00008: val_loss did not improve
 - 2s - loss: 0.2456 - acc: 0.8890 - val_loss: 0.3250 - val_acc: 0.8287
Epoch 9/120
Epoch 00009: val_loss did not improve
 - 2s - loss: 0.2830 - acc: 0.8834 - val_loss: 0.2728 - val_acc: 0.8816
Epoch 10/120
Epoch 00010: val_loss did not improve
 - 2s - loss: 0.2773 - acc: 0.8810 - val_loss: 0.2257 - val_acc: 0.8972
Epoch 11/120
Epoch 00011: val_loss improved from 0.22033 to 0.21103, saving model to best_m.h5
 - 2s - loss: 0.2649 - acc: 0.8873 - val_loss: 0.2110 - val_acc: 0.9221
Epoch 12/120
Epoch 00012: val_loss did not improve
 - 2s - loss: 0.2538 - acc: 0.8875 - val_loss: 0.3113 - val_acc: 0.8380
Epoch 13/120
Epoch 00013: val_loss did not improve
 - 2s - loss: 0.2550 - acc: 0.8930 - val_loss: 0.2632 - val_acc: 0.8723
Epoch 14/120
Epoch 00014: val_loss did not improve
 - 2s - loss: 0.2380 - acc: 0.9031 - val_loss: 

Epoch 73/120
Epoch 00073: val_loss did not improve
 - 2s - loss: 0.1063 - acc: 0.9609 - val_loss: 0.1788 - val_acc: 0.9315
Epoch 74/120
Epoch 00074: val_loss did not improve
 - 2s - loss: 0.1335 - acc: 0.9453 - val_loss: 0.1814 - val_acc: 0.9190
Epoch 75/120
Epoch 00075: val_loss did not improve
 - 2s - loss: 0.1221 - acc: 0.9500 - val_loss: 0.1759 - val_acc: 0.9283
Epoch 76/120
Epoch 00076: val_loss did not improve
 - 2s - loss: 0.1465 - acc: 0.9391 - val_loss: 0.1974 - val_acc: 0.9190
Epoch 77/120
Epoch 00077: val_loss did not improve
 - 2s - loss: 0.1283 - acc: 0.9430 - val_loss: 0.1735 - val_acc: 0.9283
Epoch 78/120
Epoch 00078: val_loss did not improve
 - 2s - loss: 0.1546 - acc: 0.9347 - val_loss: 0.1798 - val_acc: 0.9252
Epoch 79/120
Epoch 00079: val_loss did not improve
 - 2s - loss: 0.1064 - acc: 0.9586 - val_loss: 0.1781 - val_acc: 0.9221
Epoch 80/120
Epoch 00080: val_loss did not improve
 - 2s - loss: 0.1110 - acc: 0.9552 - val_loss: 0.1719 - val_acc: 0.9252
Epoch 81/120
Epo

Epoch 17/120
Epoch 00017: val_loss did not improve
 - 2s - loss: 0.2462 - acc: 0.8946 - val_loss: 0.3469 - val_acc: 0.8629
Epoch 18/120
Epoch 00018: val_loss did not improve
 - 2s - loss: 0.2411 - acc: 0.9023 - val_loss: 0.2714 - val_acc: 0.8972
Epoch 19/120
Epoch 00019: val_loss did not improve
 - 2s - loss: 0.2251 - acc: 0.8969 - val_loss: 0.3154 - val_acc: 0.8442
Epoch 20/120
Epoch 00020: val_loss did not improve
 - 2s - loss: 0.2057 - acc: 0.9125 - val_loss: 0.2032 - val_acc: 0.9252
Epoch 21/120
Epoch 00021: val_loss did not improve
 - 2s - loss: 0.2608 - acc: 0.8857 - val_loss: 0.2430 - val_acc: 0.9034
Epoch 22/120
Epoch 00022: val_loss improved from 0.19630 to 0.19157, saving model to best_m.h5
 - 2s - loss: 0.2273 - acc: 0.9039 - val_loss: 0.1916 - val_acc: 0.9315
Epoch 23/120
Epoch 00023: val_loss did not improve
 - 2s - loss: 0.2161 - acc: 0.9148 - val_loss: 0.3183 - val_acc: 0.8847
Epoch 24/120
Epoch 00024: val_loss did not improve
 - 2s - loss: 0.2081 - acc: 0.9055 - val_los

Epoch 82/120
Epoch 00082: val_loss did not improve
 - 2s - loss: 0.1203 - acc: 0.9484 - val_loss: 0.2148 - val_acc: 0.9190
Epoch 83/120
Epoch 00083: val_loss did not improve
 - 2s - loss: 0.1133 - acc: 0.9570 - val_loss: 0.2140 - val_acc: 0.9159
Epoch 84/120
Epoch 00084: val_loss did not improve
 - 2s - loss: 0.1059 - acc: 0.9547 - val_loss: 0.2177 - val_acc: 0.9190
Epoch 85/120
Epoch 00085: val_loss did not improve
 - 2s - loss: 0.1056 - acc: 0.9594 - val_loss: 0.2160 - val_acc: 0.9190
Epoch 86/120
Epoch 00086: val_loss did not improve
 - 2s - loss: 0.1072 - acc: 0.9555 - val_loss: 0.2083 - val_acc: 0.9252
Epoch 87/120
Epoch 00087: val_loss did not improve
 - 2s - loss: 0.1010 - acc: 0.9664 - val_loss: 0.2506 - val_acc: 0.9190
Epoch 88/120
Epoch 00088: val_loss did not improve
 - 2s - loss: 0.1086 - acc: 0.9555 - val_loss: 0.2166 - val_acc: 0.9221
Epoch 89/120
Epoch 00089: val_loss did not improve
 - 2s - loss: 0.1025 - acc: 0.9602 - val_loss: 0.2146 - val_acc: 0.9128
Epoch 90/120
Epo

Epoch 26/120
Epoch 00026: val_loss did not improve
 - 5s - loss: 0.1873 - acc: 0.9219 - val_loss: 0.2245 - val_acc: 0.9065
Epoch 27/120
Epoch 00027: val_loss did not improve
 - 5s - loss: 0.2482 - acc: 0.8888 - val_loss: 0.3851 - val_acc: 0.8536
Epoch 28/120
Epoch 00028: val_loss did not improve
 - 4s - loss: 0.2115 - acc: 0.9169 - val_loss: 0.2096 - val_acc: 0.9097
Epoch 29/120
Epoch 00029: val_loss did not improve
 - 4s - loss: 0.1851 - acc: 0.9234 - val_loss: 0.1888 - val_acc: 0.9128
Epoch 30/120
Epoch 00030: val_loss did not improve
 - 4s - loss: 0.2374 - acc: 0.9031 - val_loss: 0.2466 - val_acc: 0.8941
Epoch 31/120
Epoch 00031: val_loss improved from 0.16788 to 0.16678, saving model to best_m.h5
 - 5s - loss: 0.2001 - acc: 0.9133 - val_loss: 0.1668 - val_acc: 0.9252
Epoch 32/120
Epoch 00032: val_loss did not improve
 - 4s - loss: 0.1694 - acc: 0.9320 - val_loss: 0.1689 - val_acc: 0.9283
Epoch 33/120
Epoch 00033: val_loss improved from 0.16678 to 0.15930, saving model to best_m.h5


Epoch 00090: val_loss did not improve
 - 2s - loss: 0.1174 - acc: 0.9484 - val_loss: 0.1596 - val_acc: 0.9283
Epoch 91/120
Epoch 00091: val_loss did not improve
 - 2s - loss: 0.1321 - acc: 0.9516 - val_loss: 0.1601 - val_acc: 0.9315
Epoch 92/120
Epoch 00092: val_loss did not improve
 - 2s - loss: 0.1207 - acc: 0.9562 - val_loss: 0.1660 - val_acc: 0.9283
Epoch 93/120
Epoch 00093: val_loss did not improve
 - 2s - loss: 0.1063 - acc: 0.9578 - val_loss: 0.1663 - val_acc: 0.9221
Epoch 94/120
Epoch 00094: val_loss did not improve
 - 2s - loss: 0.1248 - acc: 0.9476 - val_loss: 0.1740 - val_acc: 0.9128
Epoch 95/120
Epoch 00095: val_loss did not improve
 - 2s - loss: 0.1166 - acc: 0.9474 - val_loss: 0.1612 - val_acc: 0.9283
Epoch 96/120
Epoch 00096: val_loss did not improve
 - 2s - loss: 0.1175 - acc: 0.9508 - val_loss: 0.1622 - val_acc: 0.9315
Epoch 97/120
Epoch 00097: val_loss did not improve
 - 2s - loss: 0.1139 - acc: 0.9552 - val_loss: 0.1982 - val_acc: 0.9159
Epoch 98/120
Epoch 00098: val

Epoch 34/120
Epoch 00034: val_loss did not improve
 - 2s - loss: 0.1572 - acc: 0.9414 - val_loss: 0.2528 - val_acc: 0.9094
Epoch 35/120
Epoch 00035: val_loss did not improve
 - 2s - loss: 0.1506 - acc: 0.9351 - val_loss: 0.2476 - val_acc: 0.9094
Epoch 36/120
Epoch 00036: val_loss did not improve
 - 2s - loss: 0.1401 - acc: 0.9399 - val_loss: 0.2627 - val_acc: 0.8750
Epoch 37/120
Epoch 00037: val_loss did not improve
 - 2s - loss: 0.1492 - acc: 0.9406 - val_loss: 0.2436 - val_acc: 0.9062
Epoch 38/120
Epoch 00038: val_loss did not improve
 - 2s - loss: 0.1250 - acc: 0.9516 - val_loss: 0.2532 - val_acc: 0.9094
Epoch 39/120
Epoch 00039: val_loss did not improve
 - 2s - loss: 0.1673 - acc: 0.9321 - val_loss: 0.2672 - val_acc: 0.9031
Epoch 40/120
Epoch 00040: val_loss did not improve
 - 2s - loss: 0.1418 - acc: 0.9359 - val_loss: 0.2515 - val_acc: 0.9000
Epoch 41/120
Epoch 00041: val_loss did not improve
 - 2s - loss: 0.1471 - acc: 0.9406 - val_loss: 0.2706 - val_acc: 0.8938
Epoch 42/120
Epo

Epoch 101/120
Epoch 00101: val_loss did not improve
 - 2s - loss: 0.0933 - acc: 0.9656 - val_loss: 0.2932 - val_acc: 0.8969
Epoch 102/120
Epoch 00102: val_loss did not improve
 - 2s - loss: 0.0831 - acc: 0.9664 - val_loss: 0.2842 - val_acc: 0.9062
Epoch 103/120
Epoch 00103: val_loss did not improve
 - 2s - loss: 0.0889 - acc: 0.9656 - val_loss: 0.3238 - val_acc: 0.8906
Epoch 104/120
Epoch 00104: val_loss did not improve
 - 2s - loss: 0.0904 - acc: 0.9594 - val_loss: 0.3340 - val_acc: 0.8812
Epoch 105/120
Epoch 00105: val_loss did not improve
 - 2s - loss: 0.0879 - acc: 0.9672 - val_loss: 0.2933 - val_acc: 0.9187
Epoch 106/120
Epoch 00106: val_loss did not improve
 - 2s - loss: 0.0981 - acc: 0.9672 - val_loss: 0.2980 - val_acc: 0.9000
Epoch 107/120
Epoch 00107: val_loss did not improve
 - 2s - loss: 0.0881 - acc: 0.9641 - val_loss: 0.3002 - val_acc: 0.8969
Epoch 108/120
Epoch 00108: val_loss did not improve
 - 2s - loss: 0.0812 - acc: 0.9703 - val_loss: 0.3507 - val_acc: 0.8844
Epoch 10

In [15]:
with open('../2nd_features/cnn_4_aug_1_fold5_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))



0.178242937269


In [16]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3), input_shape=(75, 75, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')


model model


In [17]:
def kfold_train(fold_cnt=3,rnd=252):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.80683, saving model to best_m.h5
 - 7s - loss: 0.6471 - acc: 0.6300 - val_loss: 0.8068 - val_acc: 0.5234
Epoch 2/120
Epoch 00002: val_loss did not improve
 - 4s - loss: 0.5096 - acc: 0.7306 - val_loss: 1.0239 - val_acc: 0.5265
Epoch 3/120
Epoch 00003: val_loss improved from 0.80683 to 0.76524, saving model to best_m.h5
 - 4s - loss: 0.4452 - acc: 0.7847 - val_loss: 0.7652 - val_acc: 0.6137
Epoch 4/120
Epoch 00004: val_loss did not improve
 - 4s - loss: 0.4172 - acc: 0.8058 - val_loss: 1.1918 - val_acc: 0.5483
Epoch 5/120
Epoch 00005: val_loss improved from 0.76524 to 0.73273, saving model to best_m.h5
 - 4s - loss: 0.3671 - acc: 0.8367 - val_loss: 0.7327 - val_acc: 0.6885
Epoch 6/120
Epoch 00006: val_loss improved from 0.73273 to 0.59475, saving model to best_m.h5
 - 3s - loss: 0.3617 - acc: 0.8414 - val_loss: 0.5948 - val_acc: 0.7196
Epoch 7/120
Epoch 00007: val_loss improved from 0.59475 to 0.33911, saving model to best_m.h5
 -

Epoch 63/120
Epoch 00063: val_loss did not improve
 - 3s - loss: 0.2112 - acc: 0.9138 - val_loss: 0.2384 - val_acc: 0.8879
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 3s - loss: 0.2148 - acc: 0.9141 - val_loss: 0.2235 - val_acc: 0.9034
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 3s - loss: 0.2038 - acc: 0.9195 - val_loss: 0.2401 - val_acc: 0.8972
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 3s - loss: 0.2083 - acc: 0.9125 - val_loss: 0.2200 - val_acc: 0.8941
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 3s - loss: 0.1902 - acc: 0.9187 - val_loss: 0.2220 - val_acc: 0.8972
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 3s - loss: 0.2071 - acc: 0.9148 - val_loss: 0.2335 - val_acc: 0.8910
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 3s - loss: 0.1953 - acc: 0.9226 - val_loss: 0.2300 - val_acc: 0.8941
Epoch 70/120
Epoch 00070: val_loss did not improve
 - 3s - loss: 0.1865 - acc: 0.9250 - val_loss: 0.2377 - val_acc: 0.9003
Epoch 71/120
Epo

Epoch 7/120
Epoch 00007: val_loss did not improve
 - 3s - loss: 0.3738 - acc: 0.8367 - val_loss: 0.4365 - val_acc: 0.7757
Epoch 8/120
Epoch 00008: val_loss improved from 0.29108 to 0.27744, saving model to best_m.h5
 - 4s - loss: 0.3494 - acc: 0.8443 - val_loss: 0.2774 - val_acc: 0.8692
Epoch 9/120
Epoch 00009: val_loss did not improve
 - 3s - loss: 0.3586 - acc: 0.8461 - val_loss: 0.3133 - val_acc: 0.8442
Epoch 10/120
Epoch 00010: val_loss did not improve
 - 3s - loss: 0.3718 - acc: 0.8240 - val_loss: 0.3438 - val_acc: 0.8318
Epoch 11/120
Epoch 00011: val_loss did not improve
 - 3s - loss: 0.3566 - acc: 0.8431 - val_loss: 0.2997 - val_acc: 0.8754
Epoch 12/120
Epoch 00012: val_loss improved from 0.27744 to 0.27051, saving model to best_m.h5
 - 3s - loss: 0.3182 - acc: 0.8615 - val_loss: 0.2705 - val_acc: 0.8816
Epoch 13/120
Epoch 00013: val_loss improved from 0.27051 to 0.21457, saving model to best_m.h5
 - 3s - loss: 0.3217 - acc: 0.8545 - val_loss: 0.2146 - val_acc: 0.9190
Epoch 14/1

Epoch 71/120
Epoch 00071: val_loss did not improve
 - 3s - loss: 0.2133 - acc: 0.9133 - val_loss: 0.1931 - val_acc: 0.9252
Epoch 72/120
Epoch 00072: val_loss did not improve
 - 3s - loss: 0.2011 - acc: 0.9187 - val_loss: 0.1844 - val_acc: 0.9315
Epoch 73/120
Epoch 00073: val_loss did not improve
 - 3s - loss: 0.2216 - acc: 0.9094 - val_loss: 0.1774 - val_acc: 0.9252
Epoch 74/120
Epoch 00074: val_loss did not improve
 - 3s - loss: 0.2177 - acc: 0.9140 - val_loss: 0.1766 - val_acc: 0.9346
Epoch 75/120
Epoch 00075: val_loss did not improve
 - 3s - loss: 0.2146 - acc: 0.9117 - val_loss: 0.1749 - val_acc: 0.9283
Epoch 76/120
Epoch 00076: val_loss did not improve
 - 3s - loss: 0.2047 - acc: 0.9172 - val_loss: 0.1838 - val_acc: 0.9252
Epoch 77/120
Epoch 00077: val_loss did not improve
 - 3s - loss: 0.2341 - acc: 0.9023 - val_loss: 0.1861 - val_acc: 0.9315
Epoch 78/120
Epoch 00078: val_loss did not improve
 - 3s - loss: 0.2073 - acc: 0.9152 - val_loss: 0.1894 - val_acc: 0.9346
Epoch 79/120
Epo

Epoch 16/120
Epoch 00016: val_loss improved from 0.23989 to 0.22877, saving model to best_m.h5
 - 4s - loss: 0.2736 - acc: 0.8773 - val_loss: 0.2288 - val_acc: 0.9003
Epoch 17/120
Epoch 00017: val_loss did not improve
 - 3s - loss: 0.2922 - acc: 0.8740 - val_loss: 0.2413 - val_acc: 0.9097
Epoch 18/120
Epoch 00018: val_loss did not improve
 - 3s - loss: 0.2808 - acc: 0.8766 - val_loss: 0.3665 - val_acc: 0.8567
Epoch 19/120
Epoch 00019: val_loss did not improve
 - 3s - loss: 0.2811 - acc: 0.8787 - val_loss: 0.3292 - val_acc: 0.8629
Epoch 20/120
Epoch 00020: val_loss did not improve
 - 3s - loss: 0.2816 - acc: 0.8766 - val_loss: 0.2780 - val_acc: 0.8879
Epoch 21/120
Epoch 00021: val_loss did not improve
 - 3s - loss: 0.2896 - acc: 0.8675 - val_loss: 0.2486 - val_acc: 0.8941
Epoch 22/120
Epoch 00022: val_loss did not improve
 - 3s - loss: 0.2666 - acc: 0.8891 - val_loss: 0.3134 - val_acc: 0.8754
Epoch 23/120
Epoch 00023: val_loss did not improve
 - 3s - loss: 0.3186 - acc: 0.8730 - val_los

Epoch 82/120
Epoch 00082: val_loss did not improve
 - 3s - loss: 0.1765 - acc: 0.9289 - val_loss: 0.2107 - val_acc: 0.9252
Epoch 83/120
Epoch 00083: val_loss did not improve
 - 3s - loss: 0.2062 - acc: 0.9162 - val_loss: 0.2397 - val_acc: 0.9097
Epoch 84/120
Epoch 00084: val_loss did not improve
 - 3s - loss: 0.1752 - acc: 0.9242 - val_loss: 0.2324 - val_acc: 0.9128
Epoch 85/120
Epoch 00085: val_loss did not improve
 - 4s - loss: 0.1729 - acc: 0.9297 - val_loss: 0.2295 - val_acc: 0.9097
Epoch 86/120
Epoch 00086: val_loss did not improve
 - 3s - loss: 0.1862 - acc: 0.9193 - val_loss: 0.2296 - val_acc: 0.9097
Epoch 87/120
Epoch 00087: val_loss did not improve
 - 3s - loss: 0.1993 - acc: 0.9115 - val_loss: 0.2183 - val_acc: 0.9065
Epoch 88/120
Epoch 00088: val_loss did not improve
 - 3s - loss: 0.1870 - acc: 0.9234 - val_loss: 0.2180 - val_acc: 0.9159
Epoch 89/120
Epoch 00089: val_loss did not improve
 - 3s - loss: 0.1882 - acc: 0.9146 - val_loss: 0.2066 - val_acc: 0.9190
Epoch 90/120
Epo

Epoch 25/120
Epoch 00025: val_loss did not improve
 - 3s - loss: 0.2826 - acc: 0.8812 - val_loss: 0.2526 - val_acc: 0.8972
Epoch 26/120
Epoch 00026: val_loss did not improve
 - 3s - loss: 0.2815 - acc: 0.8805 - val_loss: 0.2066 - val_acc: 0.9034
Epoch 27/120
Epoch 00027: val_loss did not improve
 - 3s - loss: 0.2511 - acc: 0.8914 - val_loss: 0.2223 - val_acc: 0.9097
Epoch 28/120
Epoch 00028: val_loss did not improve
 - 4s - loss: 0.2446 - acc: 0.8945 - val_loss: 0.2306 - val_acc: 0.9221
Epoch 29/120
Epoch 00029: val_loss improved from 0.19438 to 0.18459, saving model to best_m.h5
 - 3s - loss: 0.2959 - acc: 0.8745 - val_loss: 0.1846 - val_acc: 0.9346
Epoch 30/120
Epoch 00030: val_loss did not improve
 - 4s - loss: 0.2655 - acc: 0.8898 - val_loss: 0.1933 - val_acc: 0.9252
Epoch 31/120
Epoch 00031: val_loss did not improve
 - 3s - loss: 0.2371 - acc: 0.8992 - val_loss: 0.2099 - val_acc: 0.9065
Epoch 32/120
Epoch 00032: val_loss improved from 0.18459 to 0.17917, saving model to best_m.h5


Epoch 88/120
Epoch 00088: val_loss did not improve
 - 3s - loss: 0.1955 - acc: 0.9242 - val_loss: 0.1620 - val_acc: 0.9346
Epoch 89/120
Epoch 00089: val_loss did not improve
 - 3s - loss: 0.1915 - acc: 0.9234 - val_loss: 0.1631 - val_acc: 0.9252
Epoch 90/120
Epoch 00090: val_loss did not improve
 - 3s - loss: 0.1895 - acc: 0.9180 - val_loss: 0.1640 - val_acc: 0.9377
Epoch 91/120
Epoch 00091: val_loss did not improve
 - 3s - loss: 0.1917 - acc: 0.9133 - val_loss: 0.1658 - val_acc: 0.9283
Epoch 92/120
Epoch 00092: val_loss did not improve
 - 3s - loss: 0.2122 - acc: 0.9070 - val_loss: 0.1603 - val_acc: 0.9377
Epoch 93/120
Epoch 00093: val_loss did not improve
 - 3s - loss: 0.1937 - acc: 0.9180 - val_loss: 0.1592 - val_acc: 0.9315
Epoch 94/120
Epoch 00094: val_loss did not improve
 - 3s - loss: 0.1959 - acc: 0.9195 - val_loss: 0.1596 - val_acc: 0.9315
Epoch 95/120
Epoch 00095: val_loss did not improve
 - 3s - loss: 0.1748 - acc: 0.9226 - val_loss: 0.1588 - val_acc: 0.9408
Epoch 96/120
Epo

Epoch 29/120
Epoch 00029: val_loss did not improve
 - 3s - loss: 0.2381 - acc: 0.9000 - val_loss: 0.2558 - val_acc: 0.9031
Epoch 30/120
Epoch 00030: val_loss improved from 0.23580 to 0.23196, saving model to best_m.h5
 - 3s - loss: 0.2520 - acc: 0.9032 - val_loss: 0.2320 - val_acc: 0.9187
Epoch 31/120
Epoch 00031: val_loss improved from 0.23196 to 0.22509, saving model to best_m.h5
 - 3s - loss: 0.2256 - acc: 0.9047 - val_loss: 0.2251 - val_acc: 0.9219
Epoch 32/120
Epoch 00032: val_loss did not improve
 - 3s - loss: 0.1960 - acc: 0.9195 - val_loss: 0.2449 - val_acc: 0.9125
Epoch 33/120
Epoch 00033: val_loss did not improve
 - 3s - loss: 0.2148 - acc: 0.9086 - val_loss: 0.2322 - val_acc: 0.9062
Epoch 34/120
Epoch 00034: val_loss did not improve
 - 3s - loss: 0.2200 - acc: 0.8969 - val_loss: 0.2281 - val_acc: 0.9094
Epoch 35/120
Epoch 00035: val_loss did not improve
 - 3s - loss: 0.2293 - acc: 0.9101 - val_loss: 0.2307 - val_acc: 0.9219
Epoch 36/120
Epoch 00036: val_loss did not improve


Epoch 94/120
Epoch 00094: val_loss did not improve
 - 3s - loss: 0.1794 - acc: 0.9242 - val_loss: 0.2370 - val_acc: 0.9219
Epoch 95/120
Epoch 00095: val_loss did not improve
 - 4s - loss: 0.1767 - acc: 0.9305 - val_loss: 0.2348 - val_acc: 0.9219
Epoch 96/120
Epoch 00096: val_loss did not improve
 - 3s - loss: 0.2001 - acc: 0.9219 - val_loss: 0.2225 - val_acc: 0.9187
Epoch 97/120
Epoch 00097: val_loss did not improve
 - 3s - loss: 0.1908 - acc: 0.9258 - val_loss: 0.2307 - val_acc: 0.9187
Epoch 98/120
Epoch 00098: val_loss did not improve
 - 4s - loss: 0.1747 - acc: 0.9328 - val_loss: 0.2238 - val_acc: 0.9250
Epoch 99/120
Epoch 00099: val_loss did not improve
 - 4s - loss: 0.1882 - acc: 0.9258 - val_loss: 0.2224 - val_acc: 0.9250
Epoch 100/120
Epoch 00100: val_loss did not improve
 - 4s - loss: 0.1731 - acc: 0.9281 - val_loss: 0.2306 - val_acc: 0.9250
Epoch 101/120
Epoch 00101: val_loss improved from 0.21770 to 0.21503, saving model to best_m.h5
 - 3s - loss: 0.1683 - acc: 0.9344 - val_l

In [18]:
with open('../2nd_features/cnn_5_aug_1_fold5_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))



0.184668923971


In [2]:
from keras.callbacks import ModelCheckpoint
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten
from keras import layers
from keras.models import Model

def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):

    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    bn_axis = 3
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x

def create_incept_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # bn
    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
    x = conv2d_bn(x, 32, 3, 3, padding='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv2d_bn(x, 64, 1, 1, padding='valid')
    x = conv2d_bn(x, 64, 3, 3, padding='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # mixed 0, 1, 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')
    
    # mixed 1: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 96, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 96, 5, 5)

    branch3x3dbl = conv2d_bn(x, 96, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 96, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')
    
    # mixed 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 96, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 96, 5, 5)

    branch3x3dbl = conv2d_bn(x, 96, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 96, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')

    x = Conv2D(96, 3, strides=2, padding='same',activation='relu')(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')

model model


In [4]:
def kfold_train(fold_cnt=3,rnd=9):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_incept_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

import pickle
train_pred,test_pred = kfold_train(fold_cnt=5)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.69355, saving model to best_m.h5
 - 8s - loss: 0.5257 - acc: 0.7312 - val_loss: 0.6936 - val_acc: 0.4798
Epoch 2/100
Epoch 00002: val_loss improved from 0.69355 to 0.54200, saving model to best_m.h5
 - 4s - loss: 0.3957 - acc: 0.8183 - val_loss: 0.5420 - val_acc: 0.6822
Epoch 3/100
Epoch 00003: val_loss improved from 0.54200 to 0.50729, saving model to best_m.h5
 - 4s - loss: 0.3469 - acc: 0.8406 - val_loss: 0.5073 - val_acc: 0.7227
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 4s - loss: 0.3313 - acc: 0.8578 - val_loss: 0.5517 - val_acc: 0.7601
Epoch 5/100
Epoch 00005: val_loss did not improve
 - 4s - loss: 0.3269 - acc: 0.8615 - val_loss: 0.5556 - val_acc: 0.7601
Epoch 6/100
Epoch 00006: val_loss improved from 0.50729 to 0.37535, saving model to best_m.h5
 - 4s - loss: 0.3028 - acc: 0.8750 - val_loss: 0.3753 - val_acc: 0.8505
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 4s - loss: 0.2841 - acc: 0.8904 - val_loss:

Epoch 65/100
Epoch 00065: val_loss did not improve
 - 4s - loss: 0.1160 - acc: 0.9482 - val_loss: 0.4090 - val_acc: 0.8910
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 4s - loss: 0.1270 - acc: 0.9570 - val_loss: 0.3604 - val_acc: 0.8754
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 4s - loss: 0.1290 - acc: 0.9539 - val_loss: 0.3773 - val_acc: 0.8785
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 4s - loss: 0.1133 - acc: 0.9555 - val_loss: 0.3343 - val_acc: 0.8879
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 4s - loss: 0.1277 - acc: 0.9461 - val_loss: 0.3446 - val_acc: 0.8941
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 4s - loss: 0.0889 - acc: 0.9680 - val_loss: 0.3439 - val_acc: 0.8754
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 4s - loss: 0.1003 - acc: 0.9568 - val_loss: 0.3587 - val_acc: 0.8785
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 4s - loss: 0.1128 - acc: 0.9492 - val_loss: 0.3532 - val_acc: 0.8816
Epoch 73/100
Epo

Epoch 29/100
Epoch 00029: val_loss did not improve
 - 4s - loss: 0.2226 - acc: 0.9156 - val_loss: 0.1968 - val_acc: 0.9003
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 4s - loss: 0.2475 - acc: 0.8876 - val_loss: 0.2189 - val_acc: 0.8941
Epoch 31/100
Epoch 00031: val_loss did not improve
 - 4s - loss: 0.2127 - acc: 0.9109 - val_loss: 0.1646 - val_acc: 0.9252
Epoch 32/100
Epoch 00032: val_loss did not improve
 - 4s - loss: 0.1980 - acc: 0.9211 - val_loss: 0.1539 - val_acc: 0.9221
Epoch 33/100
Epoch 00033: val_loss did not improve
 - 4s - loss: 0.1953 - acc: 0.9242 - val_loss: 0.1624 - val_acc: 0.9221
Epoch 34/100
Epoch 00034: val_loss did not improve
 - 4s - loss: 0.1809 - acc: 0.9310 - val_loss: 0.1742 - val_acc: 0.9190
Epoch 35/100
Epoch 00035: val_loss did not improve
 - 4s - loss: 0.1919 - acc: 0.9226 - val_loss: 0.1581 - val_acc: 0.9252
Epoch 36/100
Epoch 00036: val_loss did not improve
 - 4s - loss: 0.1933 - acc: 0.9250 - val_loss: 0.2058 - val_acc: 0.8941
Epoch 37/100
Epo

Epoch 95/100
Epoch 00095: val_loss did not improve
 - 4s - loss: 0.1221 - acc: 0.9531 - val_loss: 0.2026 - val_acc: 0.9097
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 4s - loss: 0.1348 - acc: 0.9398 - val_loss: 0.1937 - val_acc: 0.9190
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 4s - loss: 0.0997 - acc: 0.9570 - val_loss: 0.1787 - val_acc: 0.9128
Epoch 98/100
Epoch 00098: val_loss did not improve
 - 4s - loss: 0.1104 - acc: 0.9578 - val_loss: 0.1625 - val_acc: 0.9190
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 4s - loss: 0.1244 - acc: 0.9443 - val_loss: 0.1812 - val_acc: 0.9221
Epoch 100/100
Epoch 00100: val_loss did not improve
 - 4s - loss: 0.0975 - acc: 0.9570 - val_loss: 0.1894 - val_acc: 0.9128
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.70575, saving model to best_m.h5
 - 16s - loss: 0.5605 - acc: 0.7177 - val_loss: 0.7058 - val_acc: 0.5576
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 4s - loss: 0.3804 - acc: 0.8250 - val_loss: 1

Epoch 58/100
Epoch 00058: val_loss did not improve
 - 4s - loss: 0.1318 - acc: 0.9476 - val_loss: 0.2487 - val_acc: 0.9097
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 4s - loss: 0.1215 - acc: 0.9508 - val_loss: 0.2656 - val_acc: 0.9097
Epoch 60/100
Epoch 00060: val_loss did not improve
 - 4s - loss: 0.1349 - acc: 0.9500 - val_loss: 0.2778 - val_acc: 0.9097
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 4s - loss: 0.1279 - acc: 0.9414 - val_loss: 0.2554 - val_acc: 0.9128
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 4s - loss: 0.1583 - acc: 0.9412 - val_loss: 0.2612 - val_acc: 0.9190
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 4s - loss: 0.1181 - acc: 0.9547 - val_loss: 0.2828 - val_acc: 0.9003
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 4s - loss: 0.1287 - acc: 0.9474 - val_loss: 0.2543 - val_acc: 0.9190
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 4s - loss: 0.1429 - acc: 0.9398 - val_loss: 0.2558 - val_acc: 0.9034
Epoch 66/100
Epo

Epoch 23/100
Epoch 00023: val_loss did not improve
 - 4s - loss: 0.2707 - acc: 0.8873 - val_loss: 0.2325 - val_acc: 0.9190
Epoch 24/100
Epoch 00024: val_loss did not improve
 - 4s - loss: 0.2387 - acc: 0.9000 - val_loss: 0.1544 - val_acc: 0.9408
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 4s - loss: 0.2159 - acc: 0.9148 - val_loss: 0.2402 - val_acc: 0.8785
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 4s - loss: 0.2386 - acc: 0.8988 - val_loss: 0.2776 - val_acc: 0.8692
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 4s - loss: 0.2536 - acc: 0.9055 - val_loss: 0.1951 - val_acc: 0.9221
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 4s - loss: 0.2519 - acc: 0.8959 - val_loss: 0.1792 - val_acc: 0.9283
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 4s - loss: 0.2189 - acc: 0.9125 - val_loss: 0.1531 - val_acc: 0.9315
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 4s - loss: 0.2179 - acc: 0.9109 - val_loss: 0.1958 - val_acc: 0.9252
Epoch 31/100
Epo

Epoch 89/100
Epoch 00089: val_loss did not improve
 - 4s - loss: 0.1111 - acc: 0.9568 - val_loss: 0.1289 - val_acc: 0.9595
Epoch 90/100
Epoch 00090: val_loss did not improve
 - 4s - loss: 0.1209 - acc: 0.9523 - val_loss: 0.1399 - val_acc: 0.9439
Epoch 91/100
Epoch 00091: val_loss did not improve
 - 4s - loss: 0.1234 - acc: 0.9500 - val_loss: 0.1597 - val_acc: 0.9377
Epoch 92/100
Epoch 00092: val_loss did not improve
 - 4s - loss: 0.1014 - acc: 0.9547 - val_loss: 0.1492 - val_acc: 0.9564
Epoch 93/100
Epoch 00093: val_loss did not improve
 - 4s - loss: 0.1025 - acc: 0.9545 - val_loss: 0.1474 - val_acc: 0.9315
Epoch 94/100
Epoch 00094: val_loss did not improve
 - 4s - loss: 0.1075 - acc: 0.9547 - val_loss: 0.1654 - val_acc: 0.9252
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 4s - loss: 0.1221 - acc: 0.9500 - val_loss: 0.1686 - val_acc: 0.9159
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 4s - loss: 0.1077 - acc: 0.9539 - val_loss: 0.1658 - val_acc: 0.9439
Epoch 97/100
Epo

Epoch 52/100
Epoch 00052: val_loss did not improve
 - 4s - loss: 0.1588 - acc: 0.9375 - val_loss: 0.1814 - val_acc: 0.9156
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 4s - loss: 0.1587 - acc: 0.9375 - val_loss: 0.2006 - val_acc: 0.9125
Epoch 54/100
Epoch 00054: val_loss did not improve
 - 4s - loss: 0.1594 - acc: 0.9391 - val_loss: 0.1659 - val_acc: 0.9375
Epoch 55/100
Epoch 00055: val_loss did not improve
 - 4s - loss: 0.1472 - acc: 0.9344 - val_loss: 0.1836 - val_acc: 0.9250
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 4s - loss: 0.1504 - acc: 0.9375 - val_loss: 0.1754 - val_acc: 0.9219
Epoch 57/100
Epoch 00057: val_loss did not improve
 - 4s - loss: 0.1541 - acc: 0.9391 - val_loss: 0.1878 - val_acc: 0.9062
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 4s - loss: 0.1343 - acc: 0.9461 - val_loss: 0.1694 - val_acc: 0.9125
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 4s - loss: 0.1350 - acc: 0.9484 - val_loss: 0.1950 - val_acc: 0.9062
Epoch 60/100
Epo

In [6]:
from sklearn.metrics import log_loss
with open('../2nd_features/incept_aug2_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))

# LB 1611

0.177501191209


In [7]:
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters

    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_resnet_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    bn_axis = 3
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 128], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 128], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 128], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 256], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 256], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 256], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 256], stage=3, block='d')
    
    x = Conv2D(128, 3, strides=2, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')


model model


In [8]:
def kfold_train(fold_cnt=3,rnd=9):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_resnet_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.71388, saving model to best_m.h5
 - 33s - loss: 1.0209 - acc: 0.5732 - val_loss: 0.7139 - val_acc: 0.4766
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 6s - loss: 0.5523 - acc: 0.6880 - val_loss: 0.7383 - val_acc: 0.4766
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 5s - loss: 0.4876 - acc: 0.7644 - val_loss: 0.7458 - val_acc: 0.4829
Epoch 4/100
Epoch 00004: val_loss improved from 0.71388 to 0.57105, saving model to best_m.h5
 - 6s - loss: 0.4736 - acc: 0.7777 - val_loss: 0.5710 - val_acc: 0.6511
Epoch 5/100
Epoch 00005: val_loss improved from 0.57105 to 0.38084, saving model to best_m.h5
 - 6s - loss: 0.4183 - acc: 0.8162 - val_loss: 0.3808 - val_acc: 0.8162
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 5s - loss: 0.3773 - acc: 0.8219 - val_loss: 0.4625 - val_acc: 0.7757
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 5s - loss: 0.3929 - acc: 0.8318 - val_loss: 0.6160 - val_acc: 0.8037
Epoch 8/100
Epoch

Epoch 64/100
Epoch 00064: val_loss did not improve
 - 5s - loss: 0.1730 - acc: 0.9185 - val_loss: 0.2683 - val_acc: 0.8879
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 5s - loss: 0.1816 - acc: 0.9234 - val_loss: 0.2744 - val_acc: 0.8816
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 5s - loss: 0.1570 - acc: 0.9390 - val_loss: 0.2770 - val_acc: 0.8816
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 5s - loss: 0.1738 - acc: 0.9219 - val_loss: 0.2825 - val_acc: 0.8754
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 5s - loss: 0.1815 - acc: 0.9201 - val_loss: 0.2702 - val_acc: 0.8785
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 5s - loss: 0.2000 - acc: 0.9172 - val_loss: 0.2613 - val_acc: 0.8879
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 5s - loss: 0.1572 - acc: 0.9344 - val_loss: 0.2906 - val_acc: 0.8847
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 5s - loss: 0.1750 - acc: 0.9258 - val_loss: 0.2888 - val_acc: 0.8879
Epoch 72/100
Epo

Epoch 28/100
Epoch 00028: val_loss did not improve
 - 5s - loss: 0.2714 - acc: 0.8773 - val_loss: 0.2373 - val_acc: 0.9159
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 5s - loss: 0.3173 - acc: 0.8750 - val_loss: 0.2249 - val_acc: 0.9315
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 5s - loss: 0.2997 - acc: 0.8764 - val_loss: 0.2560 - val_acc: 0.9128
Epoch 31/100
Epoch 00031: val_loss improved from 0.20187 to 0.19974, saving model to best_m.h5
 - 5s - loss: 0.2853 - acc: 0.8689 - val_loss: 0.1997 - val_acc: 0.9128
Epoch 32/100
Epoch 00032: val_loss improved from 0.19974 to 0.19498, saving model to best_m.h5
 - 5s - loss: 0.2688 - acc: 0.8969 - val_loss: 0.1950 - val_acc: 0.9252
Epoch 33/100
Epoch 00033: val_loss did not improve
 - 5s - loss: 0.2727 - acc: 0.8949 - val_loss: 0.2421 - val_acc: 0.9190
Epoch 34/100
Epoch 00034: val_loss did not improve
 - 5s - loss: 0.2423 - acc: 0.9070 - val_loss: 0.2429 - val_acc: 0.8879
Epoch 35/100
Epoch 00035: val_loss improved from 0.

Epoch 92/100
Epoch 00092: val_loss did not improve
 - 5s - loss: 0.1769 - acc: 0.9287 - val_loss: 0.1508 - val_acc: 0.9346
Epoch 93/100
Epoch 00093: val_loss did not improve
 - 5s - loss: 0.1742 - acc: 0.9242 - val_loss: 0.1826 - val_acc: 0.9034
Epoch 94/100
Epoch 00094: val_loss did not improve
 - 5s - loss: 0.2046 - acc: 0.9160 - val_loss: 0.1719 - val_acc: 0.9159
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 5s - loss: 0.1760 - acc: 0.9242 - val_loss: 0.1539 - val_acc: 0.9252
Epoch 96/100
Epoch 00096: val_loss improved from 0.14719 to 0.13892, saving model to best_m.h5
 - 5s - loss: 0.1419 - acc: 0.9398 - val_loss: 0.1389 - val_acc: 0.9377
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 5s - loss: 0.1963 - acc: 0.9209 - val_loss: 0.1572 - val_acc: 0.9315
Epoch 98/100
Epoch 00098: val_loss did not improve
 - 5s - loss: 0.1675 - acc: 0.9297 - val_loss: 0.1500 - val_acc: 0.9315
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 5s - loss: 0.1895 - acc: 0.9206 - val_los

 - 5s - loss: 0.1997 - acc: 0.9141 - val_loss: 0.2564 - val_acc: 0.8847
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 5s - loss: 0.1711 - acc: 0.9336 - val_loss: 0.2837 - val_acc: 0.8816
Epoch 54/100
Epoch 00054: val_loss did not improve
 - 5s - loss: 0.2189 - acc: 0.9097 - val_loss: 0.2893 - val_acc: 0.8816
Epoch 55/100
Epoch 00055: val_loss did not improve
 - 5s - loss: 0.1820 - acc: 0.9266 - val_loss: 0.2978 - val_acc: 0.8785
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 5s - loss: 0.1860 - acc: 0.9305 - val_loss: 0.2573 - val_acc: 0.8847
Epoch 57/100
Epoch 00057: val_loss did not improve
 - 5s - loss: 0.2014 - acc: 0.9172 - val_loss: 0.2691 - val_acc: 0.8879
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 5s - loss: 0.2115 - acc: 0.9115 - val_loss: 0.3040 - val_acc: 0.8754
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 5s - loss: 0.2019 - acc: 0.9074 - val_loss: 0.2709 - val_acc: 0.8879
Epoch 60/100
Epoch 00060: val_loss did not improve
 - 5s - loss: 0.

Epoch 16/100
Epoch 00016: val_loss did not improve
 - 5s - loss: 0.3107 - acc: 0.8726 - val_loss: 0.2135 - val_acc: 0.9159
Epoch 17/100
Epoch 00017: val_loss did not improve
 - 5s - loss: 0.3366 - acc: 0.8515 - val_loss: 0.2896 - val_acc: 0.8910
Epoch 18/100
Epoch 00018: val_loss did not improve
 - 5s - loss: 0.3569 - acc: 0.8474 - val_loss: 0.3624 - val_acc: 0.8318
Epoch 19/100
Epoch 00019: val_loss did not improve
 - 5s - loss: 0.3417 - acc: 0.8498 - val_loss: 0.3520 - val_acc: 0.8754
Epoch 20/100
Epoch 00020: val_loss did not improve
 - 5s - loss: 0.2909 - acc: 0.8755 - val_loss: 0.3200 - val_acc: 0.8629
Epoch 21/100
Epoch 00021: val_loss did not improve
 - 5s - loss: 0.3236 - acc: 0.8662 - val_loss: 0.2968 - val_acc: 0.8380
Epoch 22/100
Epoch 00022: val_loss did not improve
 - 5s - loss: 0.3056 - acc: 0.8695 - val_loss: 0.1907 - val_acc: 0.9252
Epoch 23/100
Epoch 00023: val_loss did not improve
 - 5s - loss: 0.2936 - acc: 0.8656 - val_loss: 0.2340 - val_acc: 0.9097
Epoch 24/100
Epo

Epoch 81/100
Epoch 00081: val_loss did not improve
 - 5s - loss: 0.1860 - acc: 0.9234 - val_loss: 0.1385 - val_acc: 0.9533
Epoch 82/100
Epoch 00082: val_loss did not improve
 - 5s - loss: 0.1656 - acc: 0.9289 - val_loss: 0.1412 - val_acc: 0.9470
Epoch 83/100
Epoch 00083: val_loss did not improve
 - 5s - loss: 0.1584 - acc: 0.9344 - val_loss: 0.1390 - val_acc: 0.9564
Epoch 84/100
Epoch 00084: val_loss did not improve
 - 5s - loss: 0.1778 - acc: 0.9195 - val_loss: 0.1633 - val_acc: 0.9470
Epoch 85/100
Epoch 00085: val_loss did not improve
 - 5s - loss: 0.1774 - acc: 0.9226 - val_loss: 0.1528 - val_acc: 0.9377
Epoch 86/100
Epoch 00086: val_loss did not improve
 - 5s - loss: 0.1679 - acc: 0.9320 - val_loss: 0.1456 - val_acc: 0.9502
Epoch 87/100
Epoch 00087: val_loss did not improve
 - 5s - loss: 0.1838 - acc: 0.9271 - val_loss: 0.1555 - val_acc: 0.9408
Epoch 88/100
Epoch 00088: val_loss did not improve
 - 5s - loss: 0.1599 - acc: 0.9336 - val_loss: 0.1390 - val_acc: 0.9439
Epoch 89/100
Epo

Epoch 45/100
Epoch 00045: val_loss improved from 0.18140 to 0.17699, saving model to best_m.h5
 - 5s - loss: 0.2074 - acc: 0.9195 - val_loss: 0.1770 - val_acc: 0.9344
Epoch 46/100
Epoch 00046: val_loss did not improve
 - 5s - loss: 0.2411 - acc: 0.9008 - val_loss: 0.1922 - val_acc: 0.9375
Epoch 47/100
Epoch 00047: val_loss did not improve
 - 5s - loss: 0.2417 - acc: 0.9062 - val_loss: 0.1903 - val_acc: 0.9281
Epoch 48/100
Epoch 00048: val_loss did not improve
 - 5s - loss: 0.2105 - acc: 0.9102 - val_loss: 0.1818 - val_acc: 0.9219
Epoch 49/100
Epoch 00049: val_loss did not improve
 - 5s - loss: 0.1982 - acc: 0.9148 - val_loss: 0.1783 - val_acc: 0.9187
Epoch 50/100
Epoch 00050: val_loss did not improve
 - 5s - loss: 0.2192 - acc: 0.9133 - val_loss: 0.1949 - val_acc: 0.9281
Epoch 51/100
Epoch 00051: val_loss improved from 0.17699 to 0.16991, saving model to best_m.h5
 - 5s - loss: 0.2031 - acc: 0.9140 - val_loss: 0.1699 - val_acc: 0.9437
Epoch 52/100
Epoch 00052: val_loss did not improve


In [9]:
with open('../2nd_features/resnet_aug2_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    


0.183658553937
